In [1]:
!pip install shapely
!pip install keplergl
!pip install pandas

In [2]:
from shapely.geometry import Polygon, Point
from keplergl import KeplerGl
import pandas as pd

In [3]:
bogota_csv = pd.read_csv('../csv_results/location_coordinates.csv', header=0, names=['Location', 'Latitude', 'Longitude'])

In [4]:
def load_polygons_in_map():
    bogota_map = KeplerGl(height=500)
    for location_name in bogota_csv.Location.unique():
        bogota_map.add_data(data=bogota_csv[bogota_csv.Location == location_name], name=location_name)
    return bogota_map

In [5]:
bogota_map = load_polygons_in_map()
bogota_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Usaquén': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…

In [6]:
points = pd.read_csv('../datasets/points.csv', error_bad_lines=False, index_col='id')
initial_points_shape = points.shape

points.columns = ['Latitude', 'Longitude']
points['Latitude'] = pd.to_numeric(points['Latitude'], errors='coerce').round(6)
points['Longitude'] = pd.to_numeric(points['Longitude'], errors='coerce').round(6)
points.dropna(inplace=True)

b'Skipping line 60: expected 3 fields, saw 6\nSkipping line 102: expected 3 fields, saw 5\nSkipping line 108: expected 3 fields, saw 5\n'


In [7]:
def remove_outlier(df_in):
    for col_name in ['Latitude', 'Longitude']:
        q1 = df_in[col_name].quantile(0.05)
        q3 = df_in[col_name].quantile(0.95)
        iqr = q3 - q1
        fence_low  = q1 - 1.5*iqr
        fence_high = q3 + 1.5*iqr
        df_out = df_in.loc[
            (df_in[col_name] > fence_low) & 
            (df_in[col_name] < fence_high)
        ]
    return df_out

In [8]:
points = remove_outlier(points)
final_points_shape = points.shape

In [9]:
print('initial shape:', initial_points_shape, 'final shape', final_points_shape)
points.head()

initial shape: (120, 2) final shape (111, 2)


,Latitude,Longitude
id,,
1,4.690908,-74.101481
2,4.579712,-74.083425
3,4.652541,-74.060370
4,4.717322,-74.133320
5,4.717767,-74.041595


In [10]:
points_map = KeplerGl(height=500)
points_map.add_data(data=points, name="points")
points_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'points': {'index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,…

In [11]:
bogota_map.add_data(data=points, name="points")
bogota_map

KeplerGl(data={'Usaquén': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…

In [12]:
def get_polygons_shape_objects(df):
    location_shapes = {}
    for location_name in df.Location.unique():
        data=df[df.Location == location_name]
        lat = data.Latitude.tolist()
        lon = data.Longitude.tolist()
        location_shapes[location_name] = Polygon(list(zip(lat, lon)))
    return location_shapes

In [13]:
shapes = get_polygons_shape_objects(bogota_csv)

In [14]:
def point_location(point):
    for location_name, polygon in shapes.items():
        if polygon.contains(point):
            return location_name
    return '-'

In [15]:
points['Location'] = points.apply(
    lambda row: point_location(Point(row.Latitude, row.Longitude)), axis = 1
)

In [16]:
points = points[points.Location != '-']

In [17]:
points.shape

(110, 3)

In [18]:
points.to_csv('points_location.csv')

In [19]:
bogota_map.save_to_html(file_name='bogota_map.html')
points_map.save_to_html(file_name='points_map.html')

Map saved to bogota_map.html!
Map saved to points_map.html!
